# Module 4: Train and register a machine learning model
In this module you will learn to train a machine learning model to predict the total ride duration (tripDuration) of yellow taxi trips in New York City based on various factors such as pickup and drop-off locations, distance, date, time, number of passengers, and rate code.

Once a model is trained, you will learn to register the trained model, and log hyperaparameters used and evaluation metrics using Trident's native integration with the MLflow framework.

[MLflow](https://mlflow.org/docs/latest/index.html) is an open source platform for managing the machine learning lifecycle with features like Tracking, Models, and Model Registry. MLflow is natively integrated with Trident Data Science Experience.

Please add the lakehouse you created earlier as the default lakehouse in this notebook.

#### Import mlflow and create an experiment to log the run

In [1]:
# Create Experiment to Track and register model with mlflow
import mlflow
print(f"mlflow lbrary version: {mlflow.__version__}")
EXPERIMENT_NAME = "nyctaxi_tripduration"
mlflow.set_experiment(EXPERIMENT_NAME)

StatementMeta(, 07c294f4-0823-4082-8017-42be47f9a7ac, 3, Finished, Available)

mlflow lbrary version: 2.1.1


2023/05/08 06:54:53 INFO mlflow.tracking.fluent: Experiment with name 'nyctaxi_tripduration' does not exist. Creating a new experiment.


<Experiment: artifact_location='', creation_time=1683528903111, experiment_id='803a41bb-9147-4782-bb5b-ccbc239b6beb', last_update_time=None, lifecycle_stage='active', name='nyctaxi_tripduration', tags={}>

#### Read Cleansed data from lakehouse delta table (saved in module 3)

In [2]:
SEED = 1234
# note: From the perspective of the tutorial, we are sampling training data to speed up the execution.
training_df = spark.read.format("delta").load("Tables/nyctaxi_prep").sample(fraction = 0.5, seed = SEED)

StatementMeta(, 07c294f4-0823-4082-8017-42be47f9a7ac, 4, Finished, Available)

#### Perform random split to get train and test datasets and define categorical and numeric features

In [3]:
TRAIN_TEST_SPLIT = [0.75, 0.25]
train_df, test_df = training_df.randomSplit(TRAIN_TEST_SPLIT, seed=SEED)

# Cache the dataframes to improve the speed of repeatable reads
train_df.cache()
test_df.cache()

print(f"train set count:{train_df.count()}")
print(f"test set count:{test_df.count()}")

categorical_features = ["storeAndFwdFlag","timeBins","vendorID","weekDayName","pickupHour","rateCodeId","paymentType"]
numeric_features = ['passengerCount', "tripDistance"]

StatementMeta(, 07c294f4-0823-4082-8017-42be47f9a7ac, 5, Finished, Available)

train set count:17256796
test set count:5754996


#### Define the steps to perform additional feature engineering and train the model using Spark ML pipelines and Microsoft SynapseML library
You can learn more about Spark ML pipelines [here](https://spark.apache.org/docs/latest/ml-pipeline.html), and SynapseML is documented [here](https://microsoft.github.io/SynapseML/docs/about/)

The algorithm used for this tutorial, [LightGBM](https://lightgbm.readthedocs.io/en/v3.3.2/) is a fast, distributed, high performance gradient boosting framework based on decision tree algorithms. It is an open source project developed by Microsoft and supports regression, classification and many other machine learning scenarios. Its main advantages are faster training speed, lower memory usage, better accuracy, and support for distributed learning.

In [4]:
from pyspark.ml.feature import OneHotEncoder, VectorAssembler, StringIndexer
from pyspark.ml import Pipeline
from synapse.ml.core.platform import *
from synapse.ml.lightgbm import LightGBMRegressor

# Define a pipeline steps for training a LightGBMRegressor regressor model
def lgbm_pipeline(categorical_features,numeric_features, hyperparameters):
    # String indexer
    stri = StringIndexer(inputCols=categorical_features, 
                        outputCols=[f"{feat}Idx" for feat in categorical_features]).setHandleInvalid("keep")
    # encode categorical/indexed columns
    ohe = OneHotEncoder(inputCols= stri.getOutputCols(),  
                        outputCols=[f"{feat}Enc" for feat in categorical_features])
    
    # convert all feature columns into a vector
    featurizer = VectorAssembler(inputCols=ohe.getOutputCols() + numeric_features, outputCol="features")

    # Define the LightGBM regressor
    lgr = LightGBMRegressor(
        objective = hyperparameters["objective"],
        alpha = hyperparameters["alpha"],
        learningRate = hyperparameters["learning_rate"],
        numLeaves = hyperparameters["num_leaves"],
        labelCol="tripDuration",
        numIterations = hyperparameters["iterations"],
    )
    # Define the steps and sequence of the SPark ML pipeline
    ml_pipeline = Pipeline(stages=[stri, ohe, featurizer, lgr])
    return ml_pipeline


StatementMeta(, 07c294f4-0823-4082-8017-42be47f9a7ac, 6, Finished, Available)

#### Define Training Hyperparameters
Hyperparameters are the parameters that you can change to control how a machine learning model is trained. Hyperparameters can affect the speed, quality and accuracy of the model. Some common methods to find the best hyperparameters are by testing different values, using a grid or random search, or using a more advanced optimization technique.
The hyperparameters for the lightgbm model in this tutorial have been pre-tuned using a distributed gridsearch run using [hyperopt](https://github.com/hyperopt/hyperopt)

#### Model Run 1: Using default lightgbm hyperparameters

In [5]:
# Default hyperparameters for LightGBM Model
LGBM_PARAMS = {"objective":"regression",
    "alpha":0.9,
    "learning_rate":0.1,
    "num_leaves":31,
    "iterations":100}

StatementMeta(, 07c294f4-0823-4082-8017-42be47f9a7ac, 7, Finished, Available)

#### Fit the defined pipeline on the training dataframe and generate predictions on the test dataset

In [6]:
if mlflow.active_run() is None:
    mlflow.start_run()
run = mlflow.active_run()
print(f"Active experiment run_id: {run.info.run_id}")
lg_pipeline = lgbm_pipeline(categorical_features,numeric_features,LGBM_PARAMS)
lg_model = lg_pipeline.fit(train_df)

# Get Predictions
lg_predictions = lg_model.transform(test_df)
## Caching predictions to run model evaluation faster
lg_predictions.cache()
print(f"Prediction run for {lg_predictions.count()} samples")

StatementMeta(, 07c294f4-0823-4082-8017-42be47f9a7ac, 8, Finished, Available)

Active experiment run_id: 47e80a47-a78f-45bb-925d-a8450e56ace3
Prediction run for 5754996 samples


#### Compute Model Statistics for evaluating performance of the trained LightGBMRegressor model

In [21]:
from synapse.ml.train import ComputeModelStatistics
lg_metrics = ComputeModelStatistics(
    evaluationMetric="regression", labelCol="tripDuration", scoresCol="prediction"
).transform(lg_predictions) 
display(lg_metrics)

StatementMeta(, 07c294f4-0823-4082-8017-42be47f9a7ac, 23, Finished, Available)

SynapseWidget(Synapse.DataFrame, 838a4109-46a5-43e7-92c9-e60eb76d273f)

#### Register the trained LightGBMRegressor model using MLflow

In [8]:
from mlflow.models.signature import ModelSignature 
from mlflow.types.utils import _infer_schema 

# Define a function to register a spark model
def register_spark_model(run, model, model_name,signature,metrics, hyperparameters):
        # log the model, parameters and metrics
        mlflow.spark.log_model(model, artifact_path = model_name, signature=signature, registered_model_name = model_name, dfs_tmpdir="Files/tmp/mlflow") 
        mlflow.log_params(hyperparameters) 
        mlflow.log_metrics(metrics) 
        model_uri = f"runs:/{run.info.run_id}/{model_name}" 
        print(f"Model saved in run{run.info.run_id}") 
        print(f"Model URI: {model_uri}")
        return model_uri

# Define Signature object 
sig = ModelSignature(inputs=_infer_schema(train_df.select(categorical_features + numeric_features)), 
                     outputs=_infer_schema(train_df.select("tripDuration"))) 

ALGORITHM = "lightgbm" 
model_name = f"{EXPERIMENT_NAME}_{ALGORITHM}"

# Create a 'dict' object that contains values of metrics
lg_metrics_dict = json.loads(lg_metrics.toJSON().first())

# Call model register function
model_uri = register_spark_model(run = run,
                                model = lg_model, 
                                model_name = model_name, 
                                signature = sig, 
                                metrics = lg_metrics_dict, 
                                hyperparameters = LGBM_PARAMS)
mlflow.end_run()

StatementMeta(, 07c294f4-0823-4082-8017-42be47f9a7ac, 10, Finished, Available)

/tmp/ipykernel_7705/2352378258.py:16: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  sig = ModelSignature(inputs=_infer_schema(train_df.select(categorical_features + numeric_features)),
2023/05/08 06:59:19 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp_ldwrdfe/model, flavor: spark), fall back to return 

Model saved in run47e80a47-a78f-45bb-925d-a8450e56ace3
Model URI: runs:/47e80a47-a78f-45bb-925d-a8450e56ace3/nyctaxi_tripduration_lightgbm


#### Model Run 2: Using tuned lightgbm hyperparameters and remove paymentType 
Since paymentType is usually selected at the end of a trip, we hypothize that it shouldn't be useful to predict trip duration.

In [9]:
# Tuned hyperparameters for LightGBM Model
TUNED_LGBM_PARAMS = {"objective":"regression",
    "alpha":0.08373361416254149,
    "learning_rate":0.0801709918703746,
    "num_leaves":92,
    "iterations":200}

# Remove paymentType
categorical_features.remove("paymentType")

StatementMeta(, 07c294f4-0823-4082-8017-42be47f9a7ac, 11, Finished, Available)

#### Fit the lightgbm pipeline with tuned hyperparameter on the training dataframe and generate predictions on the test dataset

In [10]:
if mlflow.active_run() is None:
    mlflow.start_run()
run = mlflow.active_run()
print(f"Active experiment run_id: {run.info.run_id}")
lg_pipeline_tn = lgbm_pipeline(categorical_features,numeric_features,TUNED_LGBM_PARAMS)
lg_model_tn = lg_pipeline_tn.fit(train_df)

# Get Predictions
lg_predictions_tn = lg_model_tn.transform(test_df)
## Caching predictions to run model evaluation faster
lg_predictions_tn.cache()
print(f"Prediction run for {lg_predictions_tn.count()} samples")

StatementMeta(, 07c294f4-0823-4082-8017-42be47f9a7ac, 12, Finished, Available)

Active experiment run_id: b1bb91e0-55cf-4302-86d5-53b8aba63d13
Prediction run for 5754996 samples


In [11]:
lg_metrics_tn = ComputeModelStatistics(
    evaluationMetric="regression", labelCol="tripDuration", scoresCol="prediction"
).transform(lg_predictions_tn)
display(lg_metrics_tn)

StatementMeta(, 07c294f4-0823-4082-8017-42be47f9a7ac, 13, Finished, Available)

SynapseWidget(Synapse.DataFrame, 11961b27-2c93-4331-9f4c-a53e994a9907)

#### Register the trained LightGBMRegressor model using MLflow

In [12]:
# Define Signature object 
sig_tn = ModelSignature(inputs=_infer_schema(train_df.select(categorical_features + numeric_features)), 
                     outputs=_infer_schema(train_df.select("tripDuration")))

# Create a 'dict' object that contains values of metrics
lg_metricstn_dict = json.loads(lg_metrics_tn.toJSON().first())

model_uri = register_spark_model(run = run,
                                model = lg_model_tn, 
                                model_name = model_name, 
                                signature = sig_tn, 
                                metrics = lg_metricstn_dict, 
                                hyperparameters = TUNED_LGBM_PARAMS)
mlflow.end_run()

StatementMeta(, 07c294f4-0823-4082-8017-42be47f9a7ac, 14, Finished, Available)

/tmp/ipykernel_7705/1750966625.py:2: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  sig_tn = ModelSignature(inputs=_infer_schema(train_df.select(categorical_features + numeric_features)),
2023/05/08 07:03:45 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpu6fqzas2/model, flavor: spark), fall back to retur

Model saved in runb1bb91e0-55cf-4302-86d5-53b8aba63d13
Model URI: runs:/b1bb91e0-55cf-4302-86d5-53b8aba63d13/nyctaxi_tripduration_lightgbm


Note: if you do not see your model artifact in the workspace, please make sure to refresh your browser.

#### You will need the below run_uri to execute the next tutorial

In [13]:
print(f"Please copy this run_uri: {model_uri}")

StatementMeta(, 07c294f4-0823-4082-8017-42be47f9a7ac, 15, Finished, Available)

Please copy this run_uri: runs:/b1bb91e0-55cf-4302-86d5-53b8aba63d13/nyctaxi_tripduration_lightgbm
